In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
# from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import BFV_Scheme.bfv as bfv
import random


In [3]:

params = bfv.Params(4, 2, 2**28, 2**80)
fv12= bfv.FV12(params)
public_key,private_key=fv12.generate_keys()


Initialised
Eval Key:0/80 Done
Eval Key:1/80 Done
Eval Key:2/80 Done
Eval Key:3/80 Done
Eval Key:4/80 Done
Eval Key:5/80 Done
Eval Key:6/80 Done
Eval Key:7/80 Done
Eval Key:8/80 Done
Eval Key:9/80 Done
Eval Key:10/80 Done
Eval Key:11/80 Done
Eval Key:12/80 Done
Eval Key:13/80 Done
Eval Key:14/80 Done
Eval Key:15/80 Done
Eval Key:16/80 Done
Eval Key:17/80 Done
Eval Key:18/80 Done
Eval Key:19/80 Done
Eval Key:20/80 Done
Eval Key:21/80 Done
Eval Key:22/80 Done
Eval Key:23/80 Done
Eval Key:24/80 Done
Eval Key:25/80 Done
Eval Key:26/80 Done
Eval Key:27/80 Done
Eval Key:28/80 Done
Eval Key:29/80 Done
Eval Key:30/80 Done
Eval Key:31/80 Done
Eval Key:32/80 Done
Eval Key:33/80 Done
Eval Key:34/80 Done
Eval Key:35/80 Done
Eval Key:36/80 Done
Eval Key:37/80 Done
Eval Key:38/80 Done
Eval Key:39/80 Done
Eval Key:40/80 Done
Eval Key:41/80 Done
Eval Key:42/80 Done
Eval Key:43/80 Done
Eval Key:44/80 Done
Eval Key:45/80 Done
Eval Key:46/80 Done
Eval Key:47/80 Done
Eval Key:48/80 Done
Eval Key:49/80 Don

In [4]:
for i in range(10):
    
    a=random.randrange(-1000,1000)
   
    print('Unencrypted:')
    print('a^2='+str(a**2))
    a_encrypted=public_key.encrypt(a)
   
    print('Encrypted:')
    print('a^2='+str(private_key.decrypt(a_encrypted**2)))

Unencrypted:
a^2=1225
Encrypted:
a^2=1225
Unencrypted:
a^2=148996
Encrypted:
a^2=148996
Unencrypted:
a^2=667489
Encrypted:
a^2=667501
Unencrypted:
a^2=915849
Encrypted:
a^2=915846
Unencrypted:
a^2=135424
Encrypted:
a^2=135411
Unencrypted:
a^2=986049
Encrypted:
a^2=986059
Unencrypted:
a^2=982081
Encrypted:
a^2=982056
Unencrypted:
a^2=74529
Encrypted:
a^2=74532
Unencrypted:
a^2=335241
Encrypted:
a^2=335248
Unencrypted:
a^2=698896
Encrypted:
a^2=698884


In [5]:
def read_dataset(url,col,scale):
  df=pd.read_csv(url)
#   df.drop("Price", axis=1, inplace=True)
  X = df.iloc[:, :col]
  col_name=df.columns[col]
  y = df[col_name]
  X_train, X_test, y_train, y_test = train_test_split(
  X,y , random_state=104,test_size=0.25, shuffle=True)
  X_train=np.asarray(X_train).astype('float32')
  X_test=np.asarray(X_test).astype('float32')
  y_train=np.asarray(y_train).astype('float32')
  y_test=np.asarray(y_test).astype('float32')
  scaler = MinMaxScaler(feature_range=(0, scale))
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  y_train=y_train.reshape(-1,1)
  y_test=y_test.reshape(-1,1)
  y_train = scaler.fit_transform(y_train)
  y_test = scaler.transform(y_test)
  y_train=y_train[:,0]
  y_test=y_test[:,0]
  print(X_test.shape)
  return X_train, X_test, y_train, y_test


In [6]:
X_train, X_test, y_train, y_test = read_dataset('../dataset/USA_Housing.csv',5,100)

(1250, 5)


In [7]:
y_train

array([15.702812, 43.560764, 88.219124, ..., 40.940823, 68.86467 ,
       54.827248], dtype=float32)

In [8]:
def custom_relu(x):
    return x*x+x
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(5,)),

    tf.keras.layers.Dense(2),
    # tf.keras.layers.Activation(custom_relu),
    tf.keras.layers.Dense(1),
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=30, batch_size=16)
loss = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.2f}")

Epoch 1/30
235/235 [==============================] - 2s 2ms/step - loss: 1836.2360
Epoch 2/30
235/235 [==============================] - 0s 2ms/step - loss: 252.2327
Epoch 3/30
235/235 [==============================] - 0s 2ms/step - loss: 176.3818
Epoch 4/30
235/235 [==============================] - 0s 1ms/step - loss: 138.4260
Epoch 5/30
235/235 [==============================] - 0s 2ms/step - loss: 114.4495
Epoch 6/30
235/235 [==============================] - 0s 1ms/step - loss: 100.2335
Epoch 7/30
235/235 [==============================] - 0s 2ms/step - loss: 91.8617
Epoch 8/30
235/235 [==============================] - 0s 2ms/step - loss: 86.1813
Epoch 9/30
235/235 [==============================] - 0s 1ms/step - loss: 81.9776
Epoch 10/30
235/235 [==============================] - 0s 2ms/step - loss: 78.7591
Epoch 11/30
235/235 [==============================] - 0s 2ms/step - loss: 76.3642
Epoch 12/30
235/235 [==============================] - 0s 2ms/step - loss: 74.5906
Epoch 

In [9]:
def create_model_weights_file(model,scale):
  i=0
  for layer in model.layers:
    if('activation' in layer.name):
      continue
    w = (layer.get_weights()[0]).T
    
    b = (layer.get_weights()[1]).T
    w=np.round_(w*scale).astype(int)
    b=np.round_(b*scale).astype(int)
    file_name = 'layer'+str(i)+'.npy'
    with open(file_name, 'wb') as f:
        np.save(f, w)
        np.save(f, b)
    i=i+1


In [10]:
create_model_weights_file(model,100)

In [11]:
class Model:
    def __init__(self,n_features,n_layers,n_layer_neurons,layer_activations,layer_weight_files):
        self.n_features=n_features
        self.n_layers=n_layers
        self.n_layer_neurons=n_layer_neurons
        self.layer_activations=layer_activations
        self.weights=[]
        self.biases=[]
    
        for wt_file in layer_weight_files:
            
            with open(wt_file, 'rb') as f:
              self.weights.append(np.load(f).tolist())
              self.biases.append(np.load(f).tolist())
              
        self.last_b=self.biases[-1][0]
            
    def __matmul(self,a,b):
        ## Input: a to be 2-D (n_layer_neurons[layer_no],n_features)and b to be 1-D list
        # Output : a 1-D list 
        output=[]
        for i in range(len(a)):
            res=0
            for j in range(len(b)):
                res=a[i][j]*b[j]+res
            output.append(res)
        return output
    
    def __matadd(self,a,b):
        ## Input: a and b to be 1-D list
        # Output : a 1-D list 
        ## For memory saving b is modified in calculations so be careful about order of inputs
        
        for i in range(len(a)):
            b[i]=b[i]+a[i]
        return b  
    
    def infer(self,x,encrypted=True):
        for i in range(self.n_layers):
            
            new_x=self.__matmul(self.weights[i],x)
            new_x=self.__matadd(self.biases[i],new_x)
            
            if(self.layer_activations[i]=='relu'):
               
                x=[t**2+t for t in new_x]
                # if(encrypted==True):
                #     temp=copy.deepcopy(new_x)
                #     for t in temp:
                #         print(private_key.decrypt(t))
                #     temp=copy.deepcopy(x)
                #     for t in temp:
                #         print(private_key.decrypt(t))
                if(encrypted==True):
                    x=[t**2+t for t in new_x]
                # else:
                #     x=[max(0,t) for t in new_x]
            else:  
                x=new_x  
        return x[0],self.last_b   
       
            
        
        
        
        

In [12]:
model=Model(30,2,[2,1],['none','none'],['layer0.npy','layer1.npy'])

In [13]:
normal_mse=0
encrypted_mse=0
limit=len(X_test)
for i in range(limit):
    x=X_test[i].tolist()
    x_encrypted=[public_key.encrypt(t) for t in x]
    normal_res,b1=model.infer(x,False)
    normal_res/=100
    normal_res=((normal_res-(b1/100))/100)+(b1/100)
    encrypted_res,b1=model.infer(x_encrypted)
    encrypted_res=private_key.decrypt(encrypted_res)/(100)
    encrypted_res=((encrypted_res-(b1/100))/100)+(b1/100)
    #print('Unencryted Inference:'+str(normal_res)+" Encrypted Inference:"+str(encrypted_res))
    # print(y_test[i])
    # print(normal_res)
    normal_mse+=(y_test[i]-normal_res)**2
    encrypted_mse+=(y_test[i]-encrypted_res)**2
print('Accuracy on unencrypted Test Data:'+str(normal_mse/limit))
print('Accuracy on encrypted test data:'+str(encrypted_mse/limit))
  

Accuracy on unencrypted Test Data:66.94426730596814
Accuracy on encrypted test data:66.94427434594108
